<a href="https://colab.research.google.com/github/ParthikB/Neural-Networks-from-Scratch-2/blob/NN-v0.1/Neural%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np


##############################################

def create_data():
  height = [130, 140, 135, 125, 120]
  weight = [35 , 30, 30, 35, 35]
  BMI    = ["Healthy", "Unhealthy", "Healthy", "Unhealthy", "Unhealthy"]


def sigmoid(z):
  return 1 / (1 + np.exp(-z))


def parameters(X):
  m = X.shape[0]
  
  W = np.ones((1, m))
  b = np.zeros((1, m))
  parameters = (W, b)

  return parameters

In [0]:
class Neuron:
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias   = bias
    
    
  def feedforward(self, X):
    
    # Weights inputs, add bias and then use activation function
    z = np.dot(self.weights, X) + self.bias
    a = sigmoid(z)
    
    return a

In [0]:
class NeuralNetwork:
  '''
      A Neural Network with:
        - 2 inputs
        - a hidden layer with 2 Neurons (h1, h2)
        - an output layer with 1 Neuron (o1)
        
  '''
  
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias    = bias
    
#   Creating the 3 Neurons we need using the Neuron class we defined before.
    self.h1 = Neuron(self.weights, self.bias)
    self.h2 = Neuron(self.weights, self.bias)
    self.o1 = Neuron(self.weights, self.bias)
    
    
  def feedforward(self, X):
    h1_hat = self.h1.feedforward(X)
    h2_hat = self.h2.feedforward(X)
    
#   y_hat --> predictions
#   Taking the outputs of the hidden layers Neurons and feeding it as inputs to the output Neuron.
    y_hat  = self.o1.feedforward(np.array([h1_hat, h2_hat]))
    
    return y_hat

In [31]:
nn = NeuralNetwork([0, 1], 0)
X = np.array([2, 3])

nn.feedforward(X)

0.7216325609518421